# Add Metadata to the model TFLite

This notebook demonstrates the process of improving a metadata writer for TensorFlow Lite models. We focus on error handling, code modularization, metadata verification, and logging for better maintainability and readability. 

The metadata writer is a Python script that generates metadata for a TensorFlow Lite model. The metadata includes information about the model's input and output tensors, and the model's input and output tensor types. The metadata is stored in a JSON file that can be used by other applications to understand the model's structure.

## Setup

First, we import the necessary libraries and set up logging for our operations.

In [1]:
# Show the TensorFlow version
!pip show tensorflow

# Show the tflite-support version
!pip show tflite-support

Name: tensorflow
Version: 2.4.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\lenovo\.virtualenvs\github_project_ml_side-f19xa0xl\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras-preprocessing, numpy, opt-einsum, protobuf, six, tensorboard, tensorflow-estimator, termcolor, typing-extensions, wheel, wrapt
Required-by: 
Name: tflite-support
Version: 0.3.1
Summary: TFLite Support is a toolkit that helps users to develop ML and deploy TFLite models onto mobile devices.
Home-page: https://www.tensorflow.org/
Author: Google, LLC.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\lenovo\.virtualenvs\github_project_ml_side-f19xa0xl\lib\site-packages
Requires: absl-py, flatbuffers, numpy, pybind11
Required-by: 


In [11]:
# Import the required libraries
import logging
import os
import json
import tensorflow as tf
from tflite_support import metadata as _metadata
from tflite_support.metadata_writers import image_classifier, writer_utils
from tflite_support import metadata_schema_py_generated as _metadata_fb

# Setup logging
logging.basicConfig(level=logging.INFO)

### Utility Functions

Here, we define utility functions for sorting classes, writing labels to a file, adding metadata to the model, and verifying the metadata.

In [3]:
def sort_classes(classes):
    """Sort classes alphabetically."""
    return sorted(classes)

In [4]:
def write_labels_to_file(label_path, classes):
    """Write the class labels to a file."""
    os.makedirs(os.path.dirname(label_path), exist_ok=True)
    content_to_write = "\n".join(classes) + "\n"
    try:
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                existing_content = f.read()
            if content_to_write == existing_content:
                logging.info("The file already exists with the same content. No need to write again.")
                return
        with open(label_path, 'w') as f:
            f.write(content_to_write)
        logging.info("Labels written to file.")
    except IOError as e:
        logging.error(f"Failed to write labels to file: {e}")

In [5]:
def add_metadata_to_model(tflite_model_path, tflite_model_with_metadata_path, label_path):
    """Add metadata to the TFLite model."""
    try:
        writer = image_classifier.MetadataWriter.create_for_inference(
            writer_utils.load_file(tflite_model_path),
            input_norm_mean=[0],
            input_norm_std=[1],
            label_file_paths=[label_path]
        )
        writer_utils.save_file(writer.populate(), tflite_model_with_metadata_path)
        logging.info("Metadata added to the model successfully.")
    except Exception as e:
        logging.error(f"Failed to add metadata to the model: {e}")

In [6]:
def verify_metadata(tflite_model_with_metadata_path):
    """Verify the metadata in the TFLite model."""
    try:
        displayer = _metadata.MetadataDisplayer.with_model_file(tflite_model_with_metadata_path)
        metadata_json = displayer.get_metadata_json()
        logging.info(f"Metadata in the model: {metadata_json}")
    except Exception as e:
        logging.error(f"Failed to verify metadata: {e}")

### Adding Metadata to the Model

With our utility functions in place, we can now add metadata to our TFLite model. We will sort our classes, write the labels to a file, add metadata to the model, and then verify the metadata has been added correctly.

In [7]:
# Define the classes
SKIN_CLASS = ['acnes', 'blackheads', 'darkspots', 'normal', 'wrinkles']
sorted_classes = sort_classes(SKIN_CLASS)

# Define the paths
LABELS_PATH = './metadata/labels.txt'
TFLITE_MODEL_PATH = './models/original_model.tflite'
TFLITE_MODEL_WITH_METADATA_PATH = './models/model_with_metadata.tflite'

In [8]:
# Write the labels to a file
write_labels_to_file(LABELS_PATH, sorted_classes)

# Add metadata to the model
add_metadata_to_model(TFLITE_MODEL_PATH, TFLITE_MODEL_WITH_METADATA_PATH, LABELS_PATH)

# Verify the metadata in the model
verify_metadata(TFLITE_MODEL_WITH_METADATA_PATH)

INFO:root:The file already exists with the same content. No need to write again.
INFO:root:Metadata added to the model successfully.
INFO:root:Metadata in the model: {
  "name": "ImageClassifier",
  "description": "Identify the most prominent object in the image from a known set of categories.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  0.0
                ],
                "std": [
                  1.0
                ]
              }
            }
          ],
          "stats": {
            "max": [
              255.0
    

## Write Metadata to the JSON File

We will write the metadata to a JSON file and verify that the metadata has been added correctly.

In [9]:
MODEL_INFO_PATH = './metadata/model_info.json'

# Define the function to write the model info to a file
def write_metadata_to_json(tflite_model_with_metadata_path, model_info_path):
    """Write the metadata from the TFLite model to a JSON file."""
    try:
        # Extract metadata from the TFLite model
        displayer = _metadata.MetadataDisplayer.with_model_file(tflite_model_with_metadata_path)
        metadata_json_str = displayer.get_metadata_json()
        metadata_dict = json.loads(metadata_json_str)
        
        # Write the metadata to a JSON file
        os.makedirs(os.path.dirname(model_info_path), exist_ok=True)
        with open(model_info_path, 'w') as json_file:
            json.dump(metadata_dict, json_file, indent=4)
        
        logging.info(f"Metadata written to JSON file: {model_info_path}")
    except Exception as e:
        logging.error(f"Failed to write metadata to JSON file: {e}")

In [12]:
# Write the metadata to the JSON file
write_metadata_to_json(TFLITE_MODEL_WITH_METADATA_PATH, MODEL_INFO_PATH)

INFO:root:Metadata written to JSON file: ./metadata/model_info.json


## Verify the Metadata in the Model

Finally, we will verify the metadata in the model by loading the model and checking the metadata.

In [16]:
# Define the function to print the metadata JSON
def print_metadata_json(tflite_model_with_metadata_path):
    """Print the metadata JSON from the TFLite model."""
    try:
        displayer = _metadata.MetadataDisplayer.with_model_file(tflite_model_with_metadata_path)
        metadata_json = displayer.get_metadata_json()
        print(f"Metadata in the model: \n\n{metadata_json}")
    except Exception as e:
        print(f"Failed to print metadata: {e}")

In [17]:
# Load the TFLite model with metadata and print the metadata JSON
TFLITE_MODEL_WITH_METADATA_PATH = './models/model_with_metadata.tflite'
print_metadata_json(TFLITE_MODEL_WITH_METADATA_PATH)

Metadata in the model: 

{
  "name": "ImageClassifier",
  "description": "Identify the most prominent object in the image from a known set of categories.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  0.0
                ],
                "std": [
                  1.0
                ]
              }
            }
          ],
          "stats": {
            "max": [
              255.0
            ],
            "min": [
              0.0
            ]
          }
        }
      ],
      "output_tensor_metadata": [
        {
  

## Conclusion

In this notebook, we create the metadata writer by adding error handling, modularizing the code, verifying the metadata, and implementing logging. 

By writing the metadata to a JSON file, we ensure that all relevant information about the TFLite model is easily accessible and can be utilized by other applications or for further analysis. This step completes our process of enhancing the metadata writer, making our TFLite model more understandable and easier to integrate into applications.

This approach enhances the maintainability and readability of the code, making it easier to manage and update in the future.